# Pharmacokinetics and Pharmacodynamics of Myelotoxicity

### Contents
- [Introduction](1_introduction.ipynb)
- [The Forward Model](2_forward_model.ipynb)
- **Exploring Data**
    - [Experimental data](#Experimental-data)
    - [Simulated data](#Simulated-data)
- [Parameter Inference](4_parameter_inference.ipynb)
- [Parameter Identifiability](5_profile_likelihoods.ipynb)
- [Model Selection](PD_naive_pooled_inference.ipynb)

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas
from Code.PK_model import PK_result
from Code.PD_model import PD_model
import plotly.figure_factory as ff


image_file = "../Images/data_explore/"

## Exploring Data

As discussed in section [Forward Model](forward_model.ipynb), many decisions made in mathematical modelling comes from observations made. Also, as will be seen in [parameter inference] and [model selection] sections, observations are required to analyse and refine the models, in order to produce reliable predictions. In this report, I will be using both simulated data, to test and refine the methods used throughout, and experimental data provided by my colleague at Roche.

### Experimental data
The experimental data used in this report is from a preclinical study on rats. The study involves dosing rats with different anti-tumour drugs at differing dose levels. Observations on drug concentrations in the blood stream and nine different blood cell counts are provided in the data.

In [2]:
df = pandas.read_csv("../Data_and_parameters/0470-2008_2018-05-09.csv")
drug_names = df.DRUG.unique()
drug_names = np.delete(drug_names, np.argwhere(drug_names=='Controls'))
df = df.sort_values(['DOSE', 'TIME'], ascending=True)
group = df.groupby('DRUG')
df_view = group.apply(lambda x: x['DOSE'].unique())
df_view = df_view.apply(pandas.Series)
df_view = df_view.replace(np.nan, '', regex=True)
df_view.columns = ['Dose 1', "Dose 2", "Dose 3"]
print(df)

             DRUG  DOSE    ID   OBS   TIME TIME_UNIT  EVID  CENS AMT TINF  \
646      Controls   0.0  5061  6.91  -48.0     hours     0     0   .    .   
654      Controls   0.0  5062  6.63  -48.0     hours     0     0   .    .   
662      Controls   0.0  5063  6.65  -48.0     hours     0     0   .    .   
670      Controls   0.0  5064  6.46  -48.0     hours     0     0   .    .   
678      Controls   0.0  5065  7.58  -48.0     hours     0     0   .    .   
...           ...   ...   ...   ...    ...       ...   ...   ...  ..  ...   
11281  Irinotecan  68.0  5164  0.03  552.0     hours     0     0   .    .   
11289  Irinotecan  68.0  5165  0.04  552.0     hours     0     0   .    .   
11297  Irinotecan  68.0  5166  0.03  552.0     hours     0     0   .    .   
11305  Irinotecan  68.0  5167  0.03  552.0     hours     0     0   .    .   
11313  Irinotecan  68.0  5168  0.02  552.0     hours     0     0   .    .   

           WT      UNIT  YTYPE               YNAME  MDV      STUDY SPECIES 

In [3]:
# df_drug = df.loc[(df['DRUG'] == drug) | (df['DRUG'] == 'Controls')]
observation_names = df.YNAME.unique()
print("Observations in dataset:\n")
print("Drug concentration (labelled as one of " + str(drug_names) +")")
for observation in observation_names:
    if observation not in drug_names:
        print(observation)

Observations in dataset:

Drug concentration (labelled as one of ['Vinorelbine' 'Docetaxel' 'Irinotecan' 'Topotecan' 'Vinflunine'])
Red blood cells
Hemoglobin
Platelets 
White blood cells
Neutrophiles absolute
Lymphocytes absolute
Monocytes absolute
Eosinophils absolute
Basophils absolute


In this report I consider the drug Docetaxel and the concentration of platelets in the blood. The PK and platelet data gathered for Docetaxel can be seen in Figure \ref{fig:DocetaxelPlateletData}.

<!-- I will take a naive-pooled approach
           % define naive-pooled
in this report when performing parameter inference and further analyses by assuming each individual can be ascribed the same set of parameters. -->

In [6]:
# Select Drug(s):
drugs = ['Vinorelbine', 'Docetaxel', 'Irinotecan', 'Topotecan', 'Vinflunine']
# Select Obsevation(s):
observations = ["Platelets "]
dose_unit = "mg/kg"

# Refine data
data_sets = {}
units = {}
for drug in drugs:
    df_drug = df.loc[((df['DRUG'] == drug) | (df['DRUG'] == 'Controls'))]
    
    patient_info = df_drug[['ID', 'DOSE', 'WT', 'AMT']].drop_duplicates()
    show = patient_info.drop(patient_info[patient_info['AMT'] == '.'].index)
    patient_info.loc[patient_info.DOSE == 0, 'AMT'] = 0
    patient_info = patient_info.drop(patient_info[patient_info['AMT'] == '.'].index).astype(
        {'WT': 'float64', 'AMT': 'float64'}
    )
    avg_weight = patient_info.mean()['WT']
    patient_info['WTM'] = patient_info['WT']/avg_weight
    
    df_drug = df_drug.drop(df_drug[df_drug['OBS'] == '.'].index)
    df_drug = df_drug.astype({'OBS': 'float64', 'DOSE': 'float64'})
    
    df_PK = df_drug.loc[(df['YNAME'] == drug)]
    data_sets[drug+"_"+drug] = df_PK
    df_PK_save = df_PK[['ID', 'DOSE', 'TIME', 'OBS']].copy()
    
    units[drug] =  df_PK['UNIT'].values[0]
    
    for observation_name in observations:
        df_PD = df_drug.loc[(df['YNAME'] == observation_name)]
        data_sets[drug+"_"+observation_name] = df_PD
        df_PD_save = df_PD[['ID', 'DOSE', 'TIME', 'OBS']].copy()
        
        units[observation_name] =  df_PD['UNIT'].values[0]
        df_PD_stats = df_PD[['TIME', 'DOSE', 'OBS']]
        df_PD_stats = df_PD_stats.groupby(["TIME", "DOSE"], as_index=False).filter(lambda x: len(x) > 1).groupby(["TIME", "DOSE"], as_index=False).agg({'OBS':['mean','std']})
        df_PD_stats.columns = ['TIME', "DOSE", 'OBS', 'std']
        data_sets[drug+"_"+observation_name+"_stats"] = df_PD_stats
        
        PD_times = df_PD.loc[(df_PD['ID'] == df_PD['ID'].values[5])].TIME.unique()
        # np.save("../Data_and_parameters/PD_real/data_times_"+drug+"_"+observation_name, PD_times)

        # df_PD_save.to_csv(path_or_buf="../Data_and_parameters/PD_real/data_refined_"+drug+"_"+observation_name+".csv", index = False)
#     df_PK_save.to_csv(path_or_buf="../Data_and_parameters/PK_real/data_refined_"+drug+".csv", index = False)
#     patient_info.to_csv(path_or_buf="../Data_and_parameters/patient_details_refined_"+drug+".csv", index = False)

for name, data in data_sets.items():
    data_sets[name] = data.rename(columns={'DOSE':'Dose, '+dose_unit,}, errors="raise")

In [7]:
# Create plot
for drug in drugs:
    for name in observations:
        x_label = "Time, hours"
        y_label = name + ", " + units[name]
        df_plot = data_sets[drug+"_"+observation_name]
        
        if name == drug:
            tick_distance = 1
        else:
            tick_distance = 48
        fig = px.scatter(
            df_plot,
            title=drug + " Data: " + name,
            x="TIME",
            y="OBS",
            color='Dose, '+dose_unit,
            size="WT",
            size_max=10,
            width=800, 
            height=500,
            hover_data=list(df_plot.columns))
        fig.update_xaxes(title_text=x_label, tick0=0, dtick=tick_distance)
        fig.update_yaxes(title_text=y_label)
        fig.show()

Lets look at the data for determining the Pharmacokinetic behaviour to see what it can tell us.

In [13]:
for drug in drugs:
    x_label = "Time, hours"
    y_label = drug + " concentration, " + units[drug]
    df_plot = data_sets[drug+"_"+drug]
    
#     dose_categories = df_drug.DOSE.unique()

    fig = px.line(
        df_plot,
        x="TIME",
        y="OBS",
        line_group="ID",
        color='Dose, '+dose_unit,
        width=500, 
        height=360,
        color_discrete_sequence=px.colors.qualitative.Plotly[1:],
        hover_data=list(df_plot.columns))
    
    fig.update_xaxes(title_text=x_label)
    fig.update_yaxes(title_text=y_label)
    fig['layout']['yaxis'].update(type="log", minor=dict(dtick='D1', showgrid=True), dtick = 1)
    fig.update_traces(mode='markers+lines')
    
    fig.write_image(image_file + drug + "_PK_traces.svg")
    fig.show()
    

We can use these models to determine what PK model to use (see the section [forward model](forward_model.ipynb)). If the dose normalised curves line up with each other and do not diverge then that implies the rates in the model are linear. Otherwise one or more of the transference rate or clearance is non-linear. Irinotecan is the only drug in the study that definately does not follow linear dynamics. The data for docetaxel does not fit on a straight line but it is also not fine grain enough to clearly distinguish between the shapes of 2, 3 or more compartment models. So to keep the parameter to data point ratio low, this drug would be best modelled by a two compartment model.

We can also look at the observations other than drug concentration to get a view on the effect of the drug

In [12]:
for drug in drugs:
    for observation_name in observations:
        df_plot = data_sets[drug+"_"+observation_name]

        x_label = "Time, hours"
        y_label = observation_name +", "+ units[observation_name]
        tick_distance = 24

        fig = px.line(
            df_plot,
            x="TIME",
            y="OBS",
            line_group="ID",
            color='Dose, '+dose_unit,
            width=500, 
            height=360,
            hover_data=list(df_plot.columns))
        
        fig.update_yaxes(title_text=y_label)
        fig.update_traces(mode='markers+lines',
            opacity=1,
                         )
        fig['layout']['xaxis'].update(
            type="linear",
            minor=dict(dtick=tick_distance, tick0=0, showgrid=True),
            tick0=0,
            dtick=3*tick_distance,
            title=dict(text=x_label)
        )
        # fig.write_image(image_file + drug +"_" + observation_name + "_traces.svg")
        fig.show()
        
        df_plot = data_sets[drug+"_"+observation_name+"_stats"]
        df_plot = df_plot.rename(columns={'TIME':x_label, 'OBS':"Mean " + y_label}, errors="raise")
#         df_plot = df_plot.drop(df_plot[df_plot['Dose, '+dose_unit]==0.0].index)
        df_plot = df_plot.astype({'Dose, '+dose_unit: 'str'})
        fig = px.scatter(
            df_plot,
            x=x_label,
            y="Mean " + y_label,
            error_y = "std",
            facet_col='Dose, '+dose_unit,
            color='Dose, '+dose_unit,
            width=900, 
            height=360,
        )

        fig['layout']['xaxis'].update(
            type="linear",
            minor=dict(dtick=tick_distance, tick0=0, showgrid=True),
            tick0=0,
            dtick=6*tick_distance,
            title=dict(text=x_label)
        )
        
        fig['layout']['xaxis2'].update(
            type="linear",
            minor=dict(dtick=tick_distance, tick0=0, showgrid=True),
            tick0=0,
            dtick=6*tick_distance,
            title=dict(text=x_label)
        )
        
        fig['layout']['xaxis3'].update(
            type="linear",
            minor=dict(dtick=tick_distance, tick0=0, showgrid=True),
            tick0=0,
            dtick=6*tick_distance,
            title=dict(text=x_label)
        )
        if len(df_plot['Dose, '+dose_unit].unique())>=4:
            fig['layout']['xaxis4'].update(
                type="linear",
                minor=dict(dtick=tick_distance, tick0=0, showgrid=True),
                tick0=0,
                dtick=6*tick_distance,
                title=dict(text=x_label)
            )
        fig.update_traces(mode='markers+lines')
        fig.update_layout(showlegend=False)
        # fig.write_image(image_file + drug +"_" + observation_name + "_mean.svg")
        fig.show()

This shows there is a lot of variability between individuals. This can be a little messy to see fully so let's only look at a subset of the individuals.

In [8]:
# Finding Random participants to graph

# Relevant Groups and their IDs
ids = {
    'Test_Group_1_ids' : np.arange(5061, 5073), # Control Ids 5061-5072
    'Test_Group_2_ids' : np.arange(5073, 5085), # Vinorelbine 5 Ids 5073-5084
    'Test_Group_3_ids' : np.arange(5085, 5097), # Vinorelbine 10 5085-5096
    'Test_Group_4_ids' : np.arange(5121, 5133), # Docetaxel 5 5121-5132
    'Test_Group_5_ids' : np.arange(5133, 5145), # Docetaxel 10 5133-5144
    'Test_Group_6_ids' : np.arange(5145, 5157), # Irinotecan 34 5145-5156
    'Test_Group_7_ids' : np.concatenate((np.array([5158,5159]), np.arange(5160, 5168))), # Irinotecan 68 5158-5159, 5160-5168
    'Test_Group_1b_ids' : np.arange(5097, 5109), # Vinorelbine 20 5097-5108
    'Test_Group_2b_ids' : np.arange(5109, 5121), # Docetaxel 15 Ids 5109-5120
    'Test_Group_3b_ids' : np.arange(5188, 5200), # Vinoflunine 10 5188-5199
    'Test_Group_4b_ids' : np.arange(5200, 5212), # Vinoflunine 20 5200-5211
    'Test_Group_9b_ids' : np.arange(5176, 5188), # Topotecan 7.5 5176-5187
    'Test_Group_10b_ids' : np.arange(5794, 5806), # Topotecan 15 5794-5805
}
#Selecting randomly from each group
num_to_select = 1
samples = np.array([])
for group in ids:
    sample_from_group = np.random.choice(ids[group], size=num_to_select, replace=False)
    samples = np.concatenate((samples,sample_from_group))
    print(group +": "+str(sample_from_group))

Test_Group_1_ids: [5071]
Test_Group_2_ids: [5080]
Test_Group_3_ids: [5091]
Test_Group_4_ids: [5124]
Test_Group_5_ids: [5136]
Test_Group_6_ids: [5148]
Test_Group_7_ids: [5163]
Test_Group_1b_ids: [5104]
Test_Group_2b_ids: [5115]
Test_Group_3b_ids: [5188]
Test_Group_4b_ids: [5204]
Test_Group_9b_ids: [5185]
Test_Group_10b_ids: [5798]


In [9]:
for drug in drugs:
    for observation_name in observations:
        df_drug = df.loc[((df['DRUG'] == drug) | (df['DRUG'] == 'Controls'))&(df['YNAME'] == observation_name)]
        df_drug = df_drug.drop(df_drug[df_drug['OBS'] == '.'].index)
        df_drug['OBS'] = df_drug['OBS'].astype(float)
        df_sample = df_drug.loc[(df['ID'].isin(samples))]

        dose_unit = df_drug['UNIT'].values[0]

        x_label = "Time, hours"
        y_label = observation_name + ", " + dose_unit
        tick_distance = 48
        dose_categories = df_sample.DOSE.unique()

        fig = px.line(
            df_sample,
            title=drug + " Data: " + observation_name,
            x="TIME",
            y="OBS",
            line_group="ID",
            color="DOSE",
            width=700, 
            height=500,
            hover_data=list(df.columns))

        fig.update_xaxes(title_text=x_label, tick0=0, dtick=tick_distance)
        fig.update_yaxes(title_text=y_label)
        fig.update_traces(mode='markers+lines')
        fig.show()

### Simulated data

The simulated data set is created from the Friberg model with combined multiplicative and constant noise. The more complex combined model is used to generate this data over the other models, as exploring this model would provide information on whether this mcan be identified over the less complex noise models, when the combined noise is known to have produced the data. The parameters used to produce the data are similar to those inferred in section section \ref{section:parameter-inference} to ensure it has similar properties to the data. This data is shown in Figure [???]. 

In [9]:
dose_amts = [0.0, 1.0, 2.0, 3.0]
n_patients = 15
dose_unit =  "mg"

PK_params = np.load("../Data_and_parameters/PK_sim/actual_params.npy")
PK_param_names = PK_params[0, :]
PK_params = PK_params[1, :].astype('float64')
PD_params = np.load("../Data_and_parameters/PD_sim/actual_params.npy")
PD_param_names = PD_params[0, :]
PD_params = PD_params[1, :].astype('float64')

# PK_params = [0.68, 2.8, 4.9, 2.3, 0.091]
# PK_param_names = ["V_c, L", "K_{cl}, L/hr", "V_1, L", "K_1, L/hr", "\sigma_{m, PK}"]
# np.save("../Data_and_parameters/PK_sim/actual_params.npy", [PK_param_names, PK_params])
PK_param_df = pandas.DataFrame([PK_params], columns=PK_param_names)

# PD_params = [983.1, 85.26, 0.44, 20, 42.43, 1, 0.13]
# PD_param_names = ["R_0, 10^3/\mu \ textrm{L}", "MTT, \ textrm{hr}", "\gamma", "S, \ textrm{L}/\ textrm{mg}", "\sigma_{c, PD}", "\eta", "\sigma_{m, PD}"]
# np.save("../Data_and_parameters/PD_sim/actual_params.npy", [PD_param_names, PD_params])
PD_param_df = pandas.DataFrame([PD_params], columns=PD_param_names)

fig =  ff.create_table(PK_param_df)
fig.update_layout(
    width=350,
    height=50,
)
fig.write_image(image_file + "PK_actual_params.svg")
fig.show()


fig =  ff.create_table(PD_param_df)
fig.update_layout(
    width=500,
    height=50,
)
fig.write_image(image_file + "PD_actual_params.svg")
fig.show()

In [11]:
# Create the Data
data_times = [0, 0.083, 0.333,  2.5, 4.666, 4.833, 5]  # To follow the times in the real data
# for n_amt, amt in enumerate(dose_amts[1:]):
#     values_no_noise = PK_result(amt, 2, PK_params[:-1], data_times)[:,0]
#     for i in range(0, n_patients):
#         noise = np.random.normal(0, PK_params[-1], len(data_times))
#         values_noisey = values_no_noise*(1+noise)
#         patient_id = (n_amt+1)*np.power(10, np.ceil(np.log10(n_patients)))+i
#         patient_data = np.transpose(np.asarray(
#             [[patient_id]*(len(data_times)-1), [amt]*(len(data_times)-1), data_times[1:], values_noisey[1:]]
#         ))
        
#         if (i==0 and amt == dose_amts[1]):
#             data = patient_data
#         else:
#             data = np.concatenate((data, patient_data))

# df_PK_sim = pandas.DataFrame(data, columns = ['ID', 'DOSE', 'TIME', 'OBS'])
# df_PK_sim.to_csv(r'../Data_and_parameters/PK_sim/sythesised_data_real_timepoints.csv', index = False)
df_PK_sim = pandas.read_csv("../Data_and_parameters/PK_sim/sythesised_data_real_timepoints.csv")

df_PK_sim["group"] = ['Simulated<br>data']*len(df_PK_sim)
x_label = "Time, hours"
y_label = "Drug concentration,<br>mg/L"
col_label = 'Dose, '+dose_unit
df_PK_sim = df_PK_sim.rename(columns={
    'DOSE':col_label, 
    'TIME':x_label, 
    'OBS':y_label
}, errors="raise")
df_PK_sim = df_PK_sim.astype({col_label: 'category'}, errors="raise")

more_times = np.linspace(0,max(data_times),1000)

# Visualise the data
fig = px.scatter(
    df_PK_sim,
    x=x_label,
    y=y_label,
    color = col_label,
    width=500, 
    height=360,
    color_discrete_sequence=px.colors.qualitative.Plotly[1:],
    hover_data=list(df_PK_sim.columns)
)


for i, amt in enumerate(dose_amts[1:]):
    
    more_values = PK_result(amt, 2, PK_params[:-1], more_times)[:,0]

    fig.add_trace(
        go.Scatter(
            x=more_times,
            y=more_values,
            legendgroup=amt,
            showlegend= False,
            mode="lines",
            line=go.scatter.Line(color=px.colors.qualitative.Plotly[i+1]),
            name="Noiseless<br>Simulation",
        ), 
#         row=1, col=i+1
    )
    fig.add_trace(
        go.Scatter(
            x=np.concatenate((more_times, more_times[::-1])),
            y=np.concatenate((
                more_values*(1+PK_params[-1]), 
                more_values[::-1]*(1-PK_params[-1])
            )),
            legendgroup=amt,
            showlegend= False,
            marker = dict(color=px.colors.qualitative.Plotly[i+1]),
            fill='toself',
            mode='lines',
            line=dict(width=0),
            name="Noise",
        ), 
#         row=1, col=i+1
    )
fig.add_trace(
    go.Scatter(
        x=[0],
        y=[0],
        legendgroup="Group",
        showlegend= True,
        legendgrouptitle_text=" ",
        mode="markers",
        marker = dict(color="black"),
        name='Simulated<br>data',
    ), 
#         row=1, col=i+1
)
fig.add_trace(
    go.Scatter(
        x=[0],
        y=[0],
        legendgroup="Group",
        showlegend= True,
        mode="lines",
        line=go.scatter.Line(color="black"),
        name="Noiseless<br>Simulation",
    ), 
#         row=1, col=i+1
)
fig.add_trace(
    go.Scatter(
        x=[0],
        y=[0],
        legendgroup="Group",
        showlegend= True,
        marker = dict(color="black"),
        fill='toself',
        mode='lines',
        line=dict(width=0),
        name="Noise",
    ), 
#         row=1, col=i+1
)
y_range = [np.log10(np.min(df_PK_sim[y_label]))-0.05, np.log10(np.max(df_PK_sim[y_label]))+0.2]
fig['layout']['yaxis'].update(type="log", minor=dict(dtick='D1', showgrid=True), dtick = 1, range=y_range)
# fig.write_image(image_file + "PK_simulation_data.svg")
fig.show()

In [12]:
# Create the PD Data
data_times = np.load("../Data_and_parameters/PD_real/data_times_Docetaxel_Platelets .npy")  # To follow the times in the real data

more_times=np.linspace(-48,max(data_times),1000)

PD_friberg = PD_model()
PD_friberg.set_PD_model("Friberg", PD_params[:-3])
more_values = {}
for n_amt, amt in enumerate(dose_amts[:]):
    PD_friberg.create_PK(amt, PK_params[:-1], np.max(data_times), num_comp=2)
    values_no_noise = PD_friberg.PD_result(data_times)[:,-1]
    more_values[amt] = PD_friberg.PD_result(more_times)[:,-1]
#     for i in range(0, n_patients):
#         noise = np.random.normal(0, 1, len(data_times))
#         values_noisey = values_no_noise+(PD_params[-3] + PD_params[-1]*np.power(values_no_noise, PD_params[-2]))*noise

#         patient_id = (n_amt)*np.power(10, np.ceil(np.log10(n_patients)))+i
#         patient_data = np.transpose(np.asarray(
#             [[patient_id]*(len(data_times)), [amt]*(len(data_times)), data_times, values_noisey]
#         ))
        
#         if (i==0 and amt == dose_amts[0]):
#             data = patient_data
#         else:
#             data = np.concatenate((data, patient_data))

# df_PD_sim = pandas.DataFrame(data, columns = ['ID', 'DOSE', 'TIME', 'OBS'])
# df_PD_sim.to_csv(r'../Data_and_parameters/PD_sim/sythesised_data_real_timepoints.csv', index = False)
df_PD_sim = pandas.read_csv("../Data_and_parameters/PD_sim/sythesised_data_real_timepoints.csv")

df_PD_sim["group"] = ['Simulated<br>data']*len(df_PD_sim)
x_label = "Time, hours"
y_label = "Blood Cell Count, 10^3/mcL"
col_label = 'Dose, '+dose_unit
df_PD_sim = df_PD_sim.rename(columns={
    'DOSE':col_label, 
    'TIME':x_label, 
    'OBS':y_label
}, errors="raise")
df_PD_sim = df_PD_sim.astype({col_label: 'category'}, errors="raise")


# Visualise the data
fig = px.scatter(
    df_PD_sim,
    x=x_label,
    y=y_label,
    facet_col= col_label,
    color = col_label,
    width=900, 
    height=360,
    color_discrete_sequence=px.colors.qualitative.Plotly[:],
    hover_data=list(df_PD_sim.columns)
)


for i, amt in enumerate(dose_amts[:]):
#     if i == 0:
#         fig['layout']['yaxis'].update(type="log", minor=dict(dtick='D1', showgrid=True), dtick = 1, range=y_range)
#     else:
#         fig['layout']['yaxis'+str(i+1)].update(type="log", minor=dict(dtick='D1', showgrid=True), dtick = 1, range=y_range)

    fig.add_trace(
        go.Scatter(
            x=more_times,
            y=more_values[amt],
            legendgroup=amt,
            showlegend= False,
            mode="lines",
            line=go.scatter.Line(color=px.colors.qualitative.Plotly[i]),
        ), 
        row=1, col=i+1
    )
    upper = more_values[amt]+(PD_params[-3] + PD_params[-1]*np.power(more_values[amt], PD_params[-2]))
    lower = more_values[amt]-(PD_params[-3] + PD_params[-1]*np.power(more_values[amt], PD_params[-2]))

    fig.add_trace(
        go.Scatter(
            x=np.concatenate((more_times, more_times[::-1])),
            y=np.concatenate((
                upper, 
                lower[::-1]
            )),
            legendgroup=amt,
            showlegend= False,
            marker = dict(color=px.colors.qualitative.Plotly[i]),
            fill='toself',
            mode='lines',
            line=dict(width=0),
        ), 
        row=1, col=i+1
    )
fig.add_trace(
    go.Scatter(
        x=[0],
        y=[0],
        legendgroup="Group",
        showlegend= True,
        legendgrouptitle_text=" ",
        mode="markers",
        marker = dict(color="black"),
        name='Simulated<br>data',
    ), 
        row=1, col=i+1
)
fig.add_trace(
    go.Scatter(
        x=[0],
        y=[0],
        legendgroup="Group",
        showlegend= True,
        mode="lines",
        line=go.scatter.Line(color="black"),
        name="Noiseless<br>Simulation",
    ), 
)
fig.add_trace(
    go.Scatter(
        x=[0],
        y=[0],
        legendgroup="Group",
        showlegend= True,
        marker = dict(color="black"),
        fill='toself',
        mode='lines',
        line=dict(width=0),
        name="Noise",
    ), 
)
y_range = [np.min(df_PD_sim[y_label])*(0.8), np.max(df_PD_sim[y_label])*(1.05)]
fig['layout']['yaxis'].update(range=y_range)
tick_distance = 24
fig['layout']['xaxis'].update(
    type="linear",
    minor=dict(dtick=tick_distance, tick0=0, showgrid=True),
    tick0=0,
    dtick=6*tick_distance,
    title=dict(text=x_label)
)

fig['layout']['xaxis2'].update(
    type="linear",
    minor=dict(dtick=tick_distance, tick0=0, showgrid=True),
    tick0=0,
    dtick=6*tick_distance,
    title=dict(text=x_label)
)

fig['layout']['xaxis3'].update(
    type="linear",
    minor=dict(dtick=tick_distance, tick0=0, showgrid=True),
    tick0=0,
    dtick=6*tick_distance,
    title=dict(text=x_label)
)
fig['layout']['xaxis4'].update(
    type="linear",
    minor=dict(dtick=tick_distance, tick0=0, showgrid=True),
    tick0=0,
    dtick=6*tick_distance,
    title=dict(text=x_label)
)
# fig.write_image(image_file + "PD_simulation_data.svg")
fig.show()

Next Notebook: [Parameter Inference](4_parameter_inference_naive.ipynb)